In [ ]:
import re #正则
import time
from selenium import webdriver #selenium
import jieba
import wordcloud
from lxml import etree #Xpath
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback #异常处理
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriver与Chrome版本兼容
import easygui as g #图形界面

option = webdriver.ChromeOptions()
option.add_argument('headless')
head={}
head['User-Agent']='Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'

def get_number(name):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    url='https://search.douban.com/music/subject_search?search_text='+name+'&cat=1003'
    driver.get(url)
    link1 = re.findall(r'<a href="https://music.douban.com/subject/(\d*)/" data-moreurl="onclick=&quot;', driver.page_source)
    return link1[0]


def get_data(url):
    data = ''
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=option)  #打开chrome浏览器
    wait = WebDriverWait(driver, 10)  #显示等待，等待是否有界面，10秒后无则报错
    driver.get(url) #获取源代码
    for i in range(25): #共25页
        print("正在爬取第{}页".format(i+1))
        html = etree.HTML(driver.page_source, etree.HTMLParser())
        for a in range(20): #每页最多有20条短评
            data += str(html.xpath('//*[@id="comments"]/ul/li['+str(a+1)+']/div[2]/p/span/text()'))
        try:
            confirm_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#content > div > div.article > div > div.paginator-wrapper > ul  > li:nth-child(3) > a')))
             #通过seletector进行定位点击的位置——后一页
                
        except: #打印被调用方的错误行及错误信息
            '''
            print ('=== STEP ERROR INFO START')
            traceback.print_exc()
            print ('=== STEP ERROR INFO END')
            '''
            return data
        else:
            confirm_btn.click() #点击这个confirm_btn按钮，进行翻页
        time.sleep(5) # 推迟执行5秒
    return data


def write_data(name,data): #转成txt文件
    f=open(name+'乐评.txt','w',encoding='utf-8')
    f.write(data)
    f.close()


def word_cloud(name):
    txt = open(name+"乐评.txt",'r',encoding='utf-8').read()
    words = jieba.lcut(txt)
    shuming=jieba.lcut(name)
    f = open(r'stoplist.txt','r',encoding='utf-8')
    stop_data = f.readlines()
    stop_words=[]
    for i in stop_data:
        a=i.replace('\n','').replace('\r','')
        stop_words.append(a)
    stop_words = stop_words+shuming
    counts = {}  #词频字典

    excludes ={
    '一个','书友','故事','自己','这个','这是','没有','不是','成为','可以','之中','群号','什么',
    '知道','本书','开始','不能','大家','来到','一切','......','验证','欢迎','但是 '
    '踏上','完本','这里','一群','你们','别人','已经','一起','之后','一次','从此','这种'
    '发现','就是','时候','不想','只是','还是','一只','...','简介','完结','如何'
    ,'后来','怎么','那个','每天','这首','我们','一天','只有','还有','部分','那么','但是'
    ,'为了','这样','他们','不要','真的','以为','不过','作者','最后','感觉','觉得','因为'
    ,'作为','理解','一本','虽然','而且','这么','这种','这本','反正','有点','非常','为什么'
    ,'读完','看到','也许','甚至','很多','多么','一直','简直','有些','应该','之前','东西'
    ,'一种','其实','对于','起来','看过','里面','比较','下去','出现','无法','真是','完全'
    ,'确实','可能','如果','一般','只能','之间','然后','从来','代替','总是','全部','因为'
    ,'所以','足够','不断','一些','大概','充满','一定','而是','当时','仿佛','如此','之外'
    ,'那些','此曲','一部','足以','多年','无比','一曲'


    }

    for word in words: #算词频
        if len(word) == 1:
            continue
        else:
            rword = word
        counts[rword] = counts.get(rword,0) + 1

    for word in stop_words : #删除停用词
        if word in counts :
            del counts[word]
    for word in excludes: #删除人工停用词
        if word in counts :
            del counts[word]
    items = list(counts.items())
    items.sort(key=lambda x:x[1],reverse=True) #排序
    counts = ''
    for i in range(100):
        word = items[i][0]
        count = items[i][1]
        for i in range(count):
            counts += word+' '
        print('{0:<10}{1:<5}'.format(word,count))
    w = wordcloud.WordCloud(font_path = 'msyh.ttc',width=800, height=400,background_color='white',collocations=False) #生成词云图
    w.generate(counts)
    w.to_file(name+'.png') #保存
    

def main():
    try:
        name = g.enterbox(msg="请输入乐名",title="评论词云生成器")
        url = 'https://music.douban.com/subject/'+get_number(name)+'/comments/hot?p=1'  #指定的音乐评论网址 p=1表示为第一页
        data = get_data(url)
        write_data(name,data)
        word_cloud(name)
        g.buttonbox(msg='',image=name+'.png' ,title='评论词云生成器', choices=(''))
        return 1
    except:
        return 0

if __name__=="__main__":
    main()


[WDM] - Cache is valid for [06/06/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - Driver found in cache [C:\Users\果壳恩\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


[WDM] - Cache is valid for [06/06/2020]
[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - Driver found in cache [C:\Users\果壳恩\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: use options instead of chrome_options


正在爬取第1页
正在爬取第2页
正在爬取第3页
正在爬取第4页
正在爬取第5页
正在爬取第6页
正在爬取第7页
正在爬取第8页
正在爬取第9页
正在爬取第10页
正在爬取第11页
正在爬取第12页
正在爬取第13页
正在爬取第14页
正在爬取第15页
正在爬取第16页
正在爬取第17页
正在爬取第18页
正在爬取第19页
正在爬取第20页
正在爬取第21页
正在爬取第22页
正在爬取第23页
正在爬取第24页
正在爬取第25页
歌词        74   
好听        73   
自卑        51   
听到        48   
李荣浩是      48   
戒烟        48   
李荣浩       48   
年少        26   
喜欢        26   
旋律        26   
循环        25   
首歌        25   
婚礼        25   
几杯        25   
那位        25   
朋友        25   
不错        25   
一首        25   
傷感        24   
充滿        24   
無奈        24   
不甘        24   
還有        24   
念念        24   
李榮浩新專     24   
風格怕       24   
做成        24   
懷舊        24   
風格        24   
不會        24   
觀望        24   
7.0       24   
飞轮        24   
完成度       24   
创作        24   
内在        24   
气质        24   
厚重        24   
突破        24   
前想        24   
折磨        24   
听众        24   
新专辑       24   
简单        24   
年龄        24   
自尊        24   
有人        24   
退而求其次     24   
我会        24   
